In [8]:
# equivalence_relation.py
# 집합 A = {1,2,3,4,5} 에 대한 관계 행렬(5x5) 입력 -> 성질 판별, 동치류 출력, 폐호 계산

from typing import List, Set

N = 5  # 집합 크기 (1..5)

def read_matrix() -> List[List[int]]:
    print("관계 행렬을 행 단위로 입력하세요. 각 행은 0/1로 공백으로 구분하여 5개 입력.")
    print("예: 1 0 1 0 0")
    mat = []
    for i in range(N):
        while True:
            row = input(f"행 {i+1}: ").strip().split()
            if len(row) != N:
                print(f"오류: {N}개의 숫자를 입력하세요.")
                continue
            try:
                vals = [1 if int(x) != 0 else 0 for x in row]
            except:
                print("오류: 0 또는 1 만 입력하세요.")
                continue
            mat.append(vals)
            break
    return mat

def print_matrix(mat: List[List[int]], title: str = "Matrix"):
    print(f"\n--- {title} ---")
    for r in mat:
        print(" ".join(str(x) for x in r))
    print("------------------")

def is_reflexive(mat: List[List[int]]) -> bool:
    for i in range(N):
        if mat[i][i] != 1:
            return False
    return True

def is_symmetric(mat: List[List[int]]) -> bool:
    for i in range(N):
        for j in range(N):
            if mat[i][j] != mat[j][i]:
                return False
    return True

def is_transitive(mat: List[List[int]]) -> bool:
    # if R[i][j] and R[j][k] then R[i][k]
    for i in range(N):
        for j in range(N):
            if mat[i][j]:
                for k in range(N):
                    if mat[j][k] and not mat[i][k]:
                        return False
    return True

def equivalence_classes(mat: List[List[int]]) -> List[Set[int]]:
    # 동치관계일 때 동치류
    classes = []
    for i in range(N):
        cls = {j+1 for j in range(N) if mat[i][j] == 1}
        classes.append(cls)

    unique = []
    seen = []
    for cls in classes:
        if cls not in seen:
            seen.append(cls)
            unique.append(cls)
    return unique

def reflexive_closure(mat: List[List[int]]) -> List[List[int]]:
    new = [row[:] for row in mat]
    for i in range(N):
        new[i][i] = 1
    return new

def symmetric_closure(mat: List[List[int]]) -> List[List[int]]:
    new = [row[:] for row in mat]
    for i in range(N):
        for j in range(N):
            if new[i][j] == 1:
                new[j][i] = 1
    return new

def transitive_closure(mat: List[List[int]]) -> List[List[int]]:
    # Warshall 알고리즘 (Boolean)
    new = [row[:] for row in mat]
    for k in range(N):
        for i in range(N):
            if new[i][k]:
                for j in range(N):
                    if new[k][j]:
                        if not new[i][j]:
                            new[i][j] = 1
    return new

def matrix_equals(a: List[List[int]], b: List[List[int]]) -> bool:
    for i in range(N):
        for j in range(N):
            if a[i][j] != b[i][j]:
                return False
    return True

def analyze_relation(mat: List[List[int]]):
    print_matrix(mat, "입력 관계 행렬")
    r = is_reflexive(mat)
    s = is_symmetric(mat)
    t = is_transitive(mat)
    print(f"반사관계: {'Yes' if r else 'No'}")
    print(f"대칭관계: {'Yes' if s else 'No'}")
    print(f"추이관계: {'Yes' if t else 'No'}")

    if r and s and t:
        print("\n=> 이 관계는 동치관계입니다.")
        classes = equivalence_classes(mat)
        print("동치류:")
        for idx, cl in enumerate(classes, 1):
            print(f"  클래스 {idx}: {sorted(list(cl))}")
    else:
        print("\n=> 이 관계는 동치관계가 아닙니다.")

def perform_closures_and_report(mat: List[List[int]]):
    print("\n== 폐포(closure) 처리 및 결과 ==")
    # Reflexive closure
    if is_reflexive(mat):
        print("이미 반사 관계입니다. (반사 폐포 불필요)")
        refl = mat
    else:
        refl = reflexive_closure(mat)
        print_matrix(mat, "반사 폐포 이전")
        print_matrix(refl, "반사 폐포 이후")

    # Symmetric closure
    if is_symmetric(refl):
        print("이미 대칭 관계입니다. (대칭 폐포 불필요)")
        symm = refl
    else:
        symm = symmetric_closure(refl)
        print_matrix(refl, "대칭 폐포 이전 (현재 행렬)")
        print_matrix(symm, "대칭 폐포 이후")

    # Transitive closure
    if is_transitive(symm):
        print("이미 추이 관계입니다. (추이 폐포 불필요)")
        trans = symm
    else:
        trans = transitive_closure(symm)
        print_matrix(symm, "추이 폐포 이전 (현재 행렬)")
        print_matrix(trans, "추이 폐포 이후")

    # 최종 행렬
    print_matrix(trans, "모든 폐포 적용 후 최종 행렬")
    print("\n최종 행렬에 대해 성질 재판별:")
    analyze_relation(trans)

    if is_reflexive(trans) and is_symmetric(trans) and is_transitive(trans):
        classes = equivalence_classes(trans)
        print("\n최종 동치류:")
        for idx, cl in enumerate(classes, 1):
            print(f"  클래스 {idx}: {sorted(list(cl))}")

def print_relation_graph(mat: List[List[int]]): # 관계 행렬을 그래프로 시각적으로 출력
    """
    추가 기능: 관계행렬을 텍스트 기반 그래프로 출력
    예: 1 -> 2, 3
    """
    print("\n### 추가 기능: 관계 그래프 출력 ###")
    for i in range(N):
        edges = []
        for j in range(N):
            if mat[i][j] == 1:
                edges.append(str(j+1))
        if edges:
            print(f"{i+1} -> {', '.join(edges)}")
        else:
            print(f"{i+1} -> (없음)")


if __name__ == "__main__":

    matrix = read_matrix()
    analyze_relation(matrix)
        # 폐포 처리(각각의 폐포 전/후 표시 및 최종 동치류 출력)
    print_relation_graph(matrix)
    perform_closures_and_report(matrix)



관계 행렬을 행 단위로 입력하세요. 각 행은 0/1로 공백으로 구분하여 5개 입력.
예: 1 0 1 0 0
행 1: 0 1 0 0 0
행 2: 0 0 1 0 0
행 3: 1 0 0 1 0
행 4: 0 0 0 0 1
행 5: 0 0 0 0 0

--- 입력 관계 행렬 ---
0 1 0 0 0
0 0 1 0 0
1 0 0 1 0
0 0 0 0 1
0 0 0 0 0
------------------
반사관계: No
대칭관계: No
추이관계: No

=> 이 관계는 동치관계가 아닙니다.

### 추가 기능: 관계 그래프 출력 ###
1 -> 2
2 -> 3
3 -> 1, 4
4 -> 5
5 -> (없음)

== 폐포(closure) 처리 및 결과 ==

--- 반사 폐포 이전 ---
0 1 0 0 0
0 0 1 0 0
1 0 0 1 0
0 0 0 0 1
0 0 0 0 0
------------------

--- 반사 폐포 이후 ---
1 1 0 0 0
0 1 1 0 0
1 0 1 1 0
0 0 0 1 1
0 0 0 0 1
------------------

--- 대칭 폐포 이전 (현재 행렬) ---
1 1 0 0 0
0 1 1 0 0
1 0 1 1 0
0 0 0 1 1
0 0 0 0 1
------------------

--- 대칭 폐포 이후 ---
1 1 1 0 0
1 1 1 0 0
1 1 1 1 0
0 0 1 1 1
0 0 0 1 1
------------------

--- 추이 폐포 이전 (현재 행렬) ---
1 1 1 0 0
1 1 1 0 0
1 1 1 1 0
0 0 1 1 1
0 0 0 1 1
------------------

--- 추이 폐포 이후 ---
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
------------------

--- 모든 폐포 적용 후 최종 행렬 ---
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
------------------

최